In [ ]:
import tensorflow as tf
import numpy as np


# In[112]:
num_classes = 513
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64

class Model:
    
    def __init__(self):
        pass
    
    def _init_and_build_net(self):
        tf.reset_default_graph()
        tf.initialize_all_variables()
        sess = tf.Session()
        self.sess = sess
        self._build_net_2()
        sess.run(tf.global_variables_initializer())
        
    def _save_network(self):
        saver = tf.train.Saver()
        saver.save(self.sess, './hangeul_cnn.ckpt')
    
    def weight_variable(self, shape):
        initial = tf.random.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    
    def bias_variable(self, shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
    
    def _restore_network(self):
        self.sess = tf.InteractiveSession()
        saver = tf.train.import_meta_graph('./hangeul_cnn.ckpt.meta')
        saver.restore(self.sess,'./hangeul_cnn.ckpt')
        
        graph = tf.get_default_graph()
        self.X =  self.sess.graph.get_tensor_by_name("Placeholder:0")
        self.Y =  self.sess.graph.get_tensor_by_name("Placeholder_1:0")
        self.W_conv1 = self.sess.graph.get_tensor_by_name("W1:0")
        self.W_conv2 = self.sess.graph.get_tensor_by_name("W2:0")
        self.W_conv3 = self.sess.graph.get_tensor_by_name("W3:0")
        self.b_conv1 = self.sess.graph.get_tensor_by_name("Variable:0")
        self.b_conv2 = self.sess.graph.get_tensor_by_name("Variable_1:0")
        self.b_conv3 = self.sess.graph.get_tensor_by_name("Variable_2:0")
        self.cross_entropy = self.sess.graph.get_tensor_by_name("Mean:0")
        self.train_step = self.sess.graph.get_operation_by_name("Adam")
        self.accuracy = self.sess.graph.get_tensor_by_name("Mean_1:0")
        self.W_fc1 = self.sess.graph.get_tensor_by_name("W4:0")
        self.b_fc1 = self.sess.graph.get_tensor_by_name("Variable_3:0")
        self.W_fc2 = self.sess.graph.get_tensor_by_name("W5:0")
        self.b_fc2 = self.sess.graph.get_tensor_by_name("Variable_4:0")
        self.y_logits = self.sess.graph.get_tensor_by_name("add_4:0")
        self.keep_prob =  self.sess.graph.get_tensor_by_name("Placeholder_2:0")
        
    def _build_net_2(self):
        self.X = tf.placeholder(tf.float32, [None, IMAGE_WIDTH*IMAGE_HEIGHT])
        self.Y = tf.placeholder(tf.float32, [None, num_classes])

        # Reshape the image back into two dimensions so we can perform convolution.
        x_image = tf.reshape(self.X, [-1, IMAGE_WIDTH, IMAGE_HEIGHT, 1])

        #self.W_conv1 = self.weight_variable([5, 5, 1, 32])
        self.W_conv1 = tf.get_variable("W1", shape=[5, 5, 1, 32], initializer=tf.contrib.layers.xavier_initializer())
        
        self.b_conv1 = self.bias_variable([32])
        x_conv1 = tf.nn.conv2d(x_image, self.W_conv1, strides=[1, 1, 1, 1],
                               padding='SAME')
        h_conv1 = tf.nn.relu(x_conv1 + self.b_conv1)

        # Max-pooling.
        h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1], padding='SAME')

        # Second convolutional layer. 64 feature maps.
        
        #self.W_conv2 = self.weight_variable([5, 5, 32, 64])
        self.W_conv2  = tf.get_variable("W2", shape=[5, 5, 32, 64], initializer=tf.contrib.layers.xavier_initializer())
        self.b_conv2 = self.bias_variable([64])
        x_conv2 = tf.nn.conv2d(h_pool1, self.W_conv2, strides=[1, 1, 1, 1],
                               padding='SAME')
        h_conv2 = tf.nn.relu(x_conv2 + self.b_conv2)

        h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1], padding='SAME')

        # Third convolutional layer. 128 feature maps.
        
        #self.W_conv3 = self.weight_variable([3, 3, 64, 128])
        self.W_conv3  = tf.get_variable("W3", shape=[3, 3, 64, 128], initializer=tf.contrib.layers.xavier_initializer())
        self.b_conv3 = self.bias_variable([128])
        x_conv3 = tf.nn.conv2d(h_pool2, self.W_conv3, strides=[1, 1, 1, 1],
                               padding='SAME')
        h_conv3 = tf.nn.relu(x_conv3 + self.b_conv3)

        h_pool3 = tf.nn.max_pool(h_conv3, ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1], padding='SAME')

        # Fully connected layer. Here we choose to have 1024 neurons in this layer.
        h_pool_flat = tf.reshape(h_pool3, [-1, 8*8*128])
        #self.W_fc1 = self.weight_variable([8*8*128, 1024])
        self.W_fc1 = tf.get_variable("W4", shape=[8*8*128, 1024], initializer=tf.contrib.layers.xavier_initializer())
        self.b_fc1 = self.bias_variable([1024])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, self.W_fc1) + self.b_fc1)

        # Dropout layer. This helps fight overfitting.
        self.keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, rate=1-self.keep_prob)

        # Classification layer.
        #self.W_fc2 = self.weight_variable([1024, num_classes])
        self.W_fc2 = tf.get_variable("W5", shape=[1024, num_classes], initializer=tf.contrib.layers.xavier_initializer())
        self.b_fc2 = self.bias_variable([num_classes])
        self.y_logits = tf.matmul(h_fc1_drop, self.W_fc2) + self.b_fc2

        # This isn't used for training, but for when using the saved model.
        tf.nn.softmax(self.y_logits)

        # Define our loss.
        self.cross_entropy = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(
                labels=tf.stop_gradient(self.Y),
                logits=self.y_logits
            )
        )

        # Define our optimizer for minimizing our loss. Here we choose a learning
        # rate of 0.0001 with AdamOptimizer. This utilizes someting
        # called the Adam algorithm, and utilizes adaptive learning rates and
        # momentum to get past saddle points.
        self.train_step = tf.train.AdamOptimizer(0.001).minimize(self.cross_entropy)

        # Define accuracy.
        correct_prediction = tf.equal(tf.argmax(self.y_logits, 1), tf.argmax(self.Y, 1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        self.accuracy = tf.reduce_mean(correct_prediction)
        print("net_complete")
        
    
    
    def result_to_str(self, x_test):
        return str(self.sess.run(tf.argmax(self.y_logits, 1), feed_dict={self.X: x_test, self.keep_prob: 1.0})[0])
    
    def predict(self, x_test):
        return self.sess.run(self.y_logits, feed_dict={self.X: x_test, self.keep_prob: 1.0})
    
    def get_accuracy(self):
        x_test = np.load('hangeul_test_input.npy')
        y_test = np.load('hangeul_test_label.npy')
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.keep_prob: 1.0})
    
    def train(self, x_data, y_data):
        return self.sess.run([self.cross_entropy, self.train_step], feed_dict={self.X: x_data, self.Y: y_data, self.keep_prob: 0.5})
    
    def training(self, epochs, input_name, label_name):
        training_epochs = epochs
        batch_size = 100
        
        train_input = np.load(input_name)
        train_label = np.load(label_name)
        
        print('Learning Started!')

        for epoch in range(training_epochs):
            avg_cost = 0
            total_batch = int(len(train_input) / batch_size)
 
            for i in range(total_batch):
                start = ((i+1) * batch_size) - batch_size
                end = ((i+1) * batch_size)
                batch_xs = train_input[start:end]
                batch_ys = train_label[start:end]
                
                #if i%100 == 0:
                #    train_accuracy = self.sess.run(self.accuracy, feed_dict={self.X: batch_xs, self.Y: batch_ys, self.keep_prob: 1.0})
                #    print("Step %d, Training Accuracy %g" %(i, float(train_accuracy)))
                c, _ = self.train(batch_xs, batch_ys)
                avg_cost += c / total_batch
 
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
 
        print('Learning Finished!')


    

In [ ]:
m1 = Model()

In [ ]:
#m1._init_and_build_net()
#m1.training(30, 'hangeul_train_input.npy', 'hangeul_train_label.npy')

In [ ]:
m1._restore_network()

In [ ]:
#print('Accuracy:', m1.get_accuracy())

In [ ]:
ORDER_FILE_PATH = "C:/DataSet/Hangeul/order.txt"
order_file = open(ORDER_FILE_PATH, 'r')
hangeul_order_list = order_file.readlines()
print(len(hangeul_order_list))
print(list(enumerate(hangeul_order_list)))

In [ ]:
import shutil
import cv2
import matplotlib.pyplot as plt

def img_to_vector():
    my_img_path = 'drawImage.png'
    temp_img_path = 'C:/DataSet/temp.png'

    shutil.copy(my_img_path, temp_img_path)

    temp_img = cv2.imread(temp_img_path, cv2.IMREAD_GRAYSCALE)

    plt.imshow(temp_img, cmap="Greys", interpolation='nearest')
    plt.show()
    
    temp_img = cv2.resize(temp_img, (64,64))
    temp_img = cv2.bitwise_not(temp_img)
    plt.imshow(temp_img, cmap="Greys", interpolation='nearest')
    plt.show()

    cv2.imwrite(temp_img_path, temp_img)
    return temp_img.reshape(-1,4096)

In [ ]:
result = list(m1.predict(img_to_vector())[0])
for i in range(5):
    index = result.index(max(result))
    print("[%s]: "%str(i+1) + "[%s]: "%hangeul_order_list[index].strip() + str(result[index]))
    result[index] = -999
print("Prediction:", m1.result_to_str(img_to_vector()))
print("Prediction:", hangeul_order_list[int(m1.result_to_str(img_to_vector()))])

In [ ]:
import socketserver
import threading
import os # os.path.getsize(path)
 
HOST = ''
PORT = 9100
lock = threading.Lock() # syncronized 동기화 진행하는 스레드 생성
 
class UserManager: # 사용자관리 및 채팅 메세지 전송을 담당하는 클래스
                   # ① 채팅 서버로 입장한 사용자의 등록
                   # ② 채팅을 종료하는 사용자의 퇴장 관리
                   # ③ 사용자가 입장하고 퇴장하는 관리
                   # ④ 사용자가 입력한 메세지를 채팅 서버에 접속한 모두에게 전송
 
    def __init__(self):
        self.users = {} # 사용자의 등록 정보를 담을 사전 {사용자 이름:(소켓,주소),...}
 
    def addUser(self, username, conn, addr): # 사용자 ID를 self.users에 추가하는 함수
        if username in self.users: # 이미 등록된 사용자라면
            conn.send('이미 등록된 사용자입니다.\n'.encode())
            return None
 
              # 새로운 사용자를 등록함
        lock.acquire() # 스레드 동기화를 막기위한 락
        self.users[username] = (conn, addr)
        lock.release() # 업데이트 후 락 해제

        self.sendMessageToAll('[%s]님이 입장했습니다. \n' %username)
        print('+++ 대화 참여자 수 [%d]' %len(self.users))

        return username
 
    def removeUser(self, username): #사용자를 제거하는 함수
        if username not in self.users:
            return

        lock.acquire()
        del self.users[username]
        lock.release()
 
        self.sendMessageToAll('[%s]님이 퇴장했습니다.' %username)
        print('--- 대화 참여자 수 [%d]' %len(self.users))
 
    def messageHandler(self, username, msg): # 전송한 msg를 처리하는 부분
        
        if msg.split('@')[0] == 'image_send_server_to_client':
            print('파일 전송 요청을 받았습니다 server -> client')
            file_name = msg.split('@')[1].strip()
            file_size = str(os.path.getsize(file_name))
            print('파일 이름 : ' + file_name)
            print('파일 길이 : ' + file_size)
            #if not exists(file_name): # 파일이 해당 디렉터리에 존재하지 않으면
            #    return # handle()함수를 빠져 나온다.
            self.sendMessageToAll('image_send_server_to_client@stoc_image.jpg@' + file_size + "\n")
            data_transferred = 0
            with open(file_name, 'rb') as f:
                try:
                    data = f.read(1024) # 파일을 1024바이트 읽음
                    while data: # 파일이 빈 문자열일때까지 반복
                        data_transferred += self.sendFileToAll(data)
                        #self.sendMessageToAll(msg)
                        data = f.read(1024)
                except Exception as e:
                    print(e)

            print('전송완료[%s], 전송량[%d]' %(file_name,data_transferred))
    
        else: #msg[0] != '/': # 보낸 메세지의 첫문자가 '/'가 아니면
            self.sendMessageToAll('[%s] %s' %(username, msg))
            return
 
        #if msg.strip() == '/quit': # 보낸 메세지가 'quit'이면
        #    self.removeUser(username)
        #    return -1
 
    def sendMessageToAll(self, msg):
        for conn, addr in self.users.values():
            print('뿌린 메시지 : ' + msg ,end='')
            conn.send(msg.encode())
    
    def sendFileToAll(self, msg):
        total = 0
        for conn, addr in self.users.values():
            print('전송중 : ' + str(msg))
            total += conn.send(msg)
        return total
    
           
 
class MyTcpHandler(socketserver.BaseRequestHandler):
    userman = UserManager()
     
    def handle(self): # 클라이언트가 접속시 클라이언트 주소 출력
        print('[%s] 연결됨' %self.client_address[0])
 
        try:
            username = self.registerUsername()
            msg = self.request.recv(1024)
            while msg:
                msg = msg.decode()
                if msg.split('@')[0] == 'image_send_client_to_server':
                    print('파일 전송 요청을 받았습니다 client -> server')
                    file_name = msg.split('@')[1].strip()
                    file_length_str = msg.split('@')[2].strip()
                    file_length_int = int(file_length_str)
                    print('파일 이름 : ' + file_name)
                    print('파일 길이 : ' + file_length_str)
                    file_size_count = 0
                   
                    with open(file_name, 'wb') as f:
                        try:
                            data = self.request.recv(1024)
                            while data:
                                #print('수신중 : ' + str(data))
                                file_size_count += f.write(data)
                                print('받은 데이터 길이 : ' + str(file_size_count))
                                if file_size_count >= file_length_int:
                                    break
                                data = self.request.recv(1024)
                            print('파일 수신 완료 : ' + str(file_size_count))
                        except Exception as e:
                            print(e)

                else: 
                    if self.userman.messageHandler(username, msg) == -1:
                        self.request.close()
                        break
                msg = self.request.recv(1024)
                 
        except Exception as e:
            print(e)
 
        print('[%s] 접속종료' %self.client_address[0])
        self.userman.removeUser(username)
 
    def registerUsername(self):
        while True:
            username = self.request.recv(1024)
            username = username.decode().strip()
            if self.userman.addUser(username, self.request, self.client_address):
                return username
 

class ChatingServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
        pass
         
def runServer():
    print('+++ 채팅 서버를 시작합니다.')
    print('+++ 채팅 서버를 끝내려면 Ctrl-C를 누르세요.')
 
    try:
        server = ChatingServer((HOST, PORT), MyTcpHandler)
        server.serve_forever()
    except KeyboardInterrupt:
        print('--- 채팅 서버를 종료합니다.')
        server.shutdown()
        server.server_close()

        
runServer()

+++ 채팅 서버를 시작합니다.
+++ 채팅 서버를 끝내려면 Ctrl-C를 누르세요.
[172.30.1.18] 연결됨
뿌린 메시지 : [yc]님이 입장했습니다. 
+++ 대화 참여자 수 [1]
뿌린 메시지 : [yc] ttc
뿌린 메시지 : [yc] fgg
파일 전송 요청을 받았습니다 server -> client
파일 이름 : stoc_image.jpg
파일 길이 : 4720
뿌린 메시지 : image_send_server_to_client@stoc_image.jpg@4720
전송중 : b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x13\x12\x12\x15\x12\x12\x12\x12\x15\x10\x12\x15\x12\x15\x16\x10\x15\x15\x10\x15\x15\x10\x15\x15\x12\x16\x16\x15\x15\x15\x15\x18\x1d( \x18\x1a%\x1b\x15\x15!1!%)+...\x17\x1f3:3-7(9.+\x01\n\n\n\x0e\r\x0e\x1a\x10\x10\x17+\x1d\x1d\x1d-.---+-+---------+-.---------++-------------------\xff\xc0\x00\x11\x08\x00\xb4\x01\x17\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x03\x00\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x04\x06\x07\x05\x03\x08\xff\xc4\x00I\x10\x00\x02\x01\x02\x03\x05\x05\x05\x03\x07\x08\n\x03\x00\x00\x00\x00\x01\x02\x03\x11\x04\x121\x05!AQa\x07\x13q\x81\x91\x06\xa1\xb1\xc1\x

파일 전송 요청을 받았습니다 client -> server
파일 이름 : ctos_image.jpg
파일 길이 : 700318
받은 데이터 길이 : 1024
받은 데이터 길이 : 2048
받은 데이터 길이 : 3072
받은 데이터 길이 : 4096
받은 데이터 길이 : 5120
받은 데이터 길이 : 6144
받은 데이터 길이 : 7168
받은 데이터 길이 : 8192
받은 데이터 길이 : 9216
받은 데이터 길이 : 10220
받은 데이터 길이 : 11244
받은 데이터 길이 : 12268
받은 데이터 길이 : 13140
받은 데이터 길이 : 14164
받은 데이터 길이 : 15188
받은 데이터 길이 : 16212
받은 데이터 길이 : 17236
받은 데이터 길이 : 18260
받은 데이터 길이 : 19284
받은 데이터 길이 : 20308
받은 데이터 길이 : 21332
받은 데이터 길이 : 22356
받은 데이터 길이 : 23380
받은 데이터 길이 : 24404
받은 데이터 길이 : 24820
받은 데이터 길이 : 25844
받은 데이터 길이 : 26868
받은 데이터 길이 : 27740
받은 데이터 길이 : 28764
받은 데이터 길이 : 29788
받은 데이터 길이 : 30812
받은 데이터 길이 : 31836
받은 데이터 길이 : 32860
받은 데이터 길이 : 33580
받은 데이터 길이 : 34604
받은 데이터 길이 : 35628
받은 데이터 길이 : 36652
받은 데이터 길이 : 37676
받은 데이터 길이 : 38700
받은 데이터 길이 : 39724
받은 데이터 길이 : 40748
받은 데이터 길이 : 41772
받은 데이터 길이 : 42796
받은 데이터 길이 : 43820
받은 데이터 길이 : 44844
받은 데이터 길이 : 45260
받은 데이터 길이 : 46284
받은 데이터 길이 : 47308
받은 데이터 길이 : 48332
받은 데이터 길이 : 49356
받은 데이터 길이 : 50380
받은 데이터 길이 : 51404
받은

파일 전송 요청을 받았습니다 server -> client
파일 이름 : stoc_image.jpg
파일 길이 : 4720
뿌린 메시지 : image_send_server_to_client@stoc_image.jpg@4720
전송중 : b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x13\x12\x12\x15\x12\x12\x12\x12\x15\x10\x12\x15\x12\x15\x16\x10\x15\x15\x10\x15\x15\x10\x15\x15\x12\x16\x16\x15\x15\x15\x15\x18\x1d( \x18\x1a%\x1b\x15\x15!1!%)+...\x17\x1f3:3-7(9.+\x01\n\n\n\x0e\r\x0e\x1a\x10\x10\x17+\x1d\x1d\x1d-.---+-+---------+-.---------++-------------------\xff\xc0\x00\x11\x08\x00\xb4\x01\x17\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x03\x00\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x04\x06\x07\x05\x03\x08\xff\xc4\x00I\x10\x00\x02\x01\x02\x03\x05\x05\x05\x03\x07\x08\n\x03\x00\x00\x00\x00\x01\x02\x03\x11\x04\x121\x05!AQa\x07\x13q\x81\x91\x06\xa1\xb1\xc1\xf0\x14"\xd1\x15#2rs\xb4\xf1BR\x82\x84\x93\xb2\xb3\xe1$%&56CTt\xc2\xd2\x1734\xff\xc4\x00\x1a\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\

파일 전송 요청을 받았습니다 server -> client
파일 이름 : stoc_image.jpg
파일 길이 : 4720
뿌린 메시지 : image_send_server_to_client@stoc_image.jpg@4720
전송중 : b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x13\x12\x12\x15\x12\x12\x12\x12\x15\x10\x12\x15\x12\x15\x16\x10\x15\x15\x10\x15\x15\x10\x15\x15\x12\x16\x16\x15\x15\x15\x15\x18\x1d( \x18\x1a%\x1b\x15\x15!1!%)+...\x17\x1f3:3-7(9.+\x01\n\n\n\x0e\r\x0e\x1a\x10\x10\x17+\x1d\x1d\x1d-.---+-+---------+-.---------++-------------------\xff\xc0\x00\x11\x08\x00\xb4\x01\x17\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x03\x00\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x04\x06\x07\x05\x03\x08\xff\xc4\x00I\x10\x00\x02\x01\x02\x03\x05\x05\x05\x03\x07\x08\n\x03\x00\x00\x00\x00\x01\x02\x03\x11\x04\x121\x05!AQa\x07\x13q\x81\x91\x06\xa1\xb1\xc1\xf0\x14"\xd1\x15#2rs\xb4\xf1BR\x82\x84\x93\xb2\xb3\xe1$%&56CTt\xc2\xd2\x1734\xff\xc4\x00\x1a\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\

In [ ]:
            if file_name == darwImage.png:
                result = list(m1.predict(img_to_vector())[0])
                for i in range(5):
                    index = result.index(max(result))
                    print("[%s]: "%str(i+1) + "[%s]: "%hangeul_order_list[index].strip() + str(result[index]))
                    result[index] = -999
                print("Prediction:", m1.result_to_str(img_to_vector()))
                print("Prediction:", hangeul_order_list[int(m1.result_to_str(img_to_vector()))])